In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from delta.tables import *
from pyspark.sql.window import Window

In [0]:
orders_df = spark.read.format('parquet').option('header','true').option('path','abfss://bronze@adls5485.dfs.core.windows.net/orders').load()

In [0]:
orders_df.display()

In [0]:
df = orders_df.withColumnRenamed('_rescued_data','Rescued_Data')
df.display()

In [0]:
df = df.withColumn('order_date',to_date(col('order_date'),'MM-dd-yyyy'))
df.display()

In [0]:
df = df.withColumn('month',month(col('order_date')))
df = df.withColumn('year',year(col('order_date')))
df.display()


In [0]:
df1 = df.withColumn('flag',dense_rank().over(Window.partitionBy('year').orderBy(col('total_amount').desc())))
df2 = df1.withColumn('flag_rank',rank().over(Window.partitionBy('total_amount').orderBy(col('year').desc())))
df3 = df2.withColumn('flag_row_number',row_number().over(Window.partitionBy('total_amount').orderBy(col('year').desc())))
df1.display()
df2.display()
df3.display()

In [0]:
df_new = df

In [0]:
class windows:
    def dense_rank(self,df_new):
        df_dense_rank = df_new.withColumn('dense_rank',dense_rank().over(Window.partitionBy('year').orderBy('total_amount')))
        return df_dense_rank
    def rank(self,df_new):
        df_rank = df_new.withColumn('rank',rank().over(Window.partitionBy('total_amount').orderBy('year')))
        return df_rank
    def row_number(self,df):
        df_row_number = df_new.withColumn('row_number',row_number().over(Window.partitionBy('total_amount').orderBy('year')))
        return df_row_number

In [0]:
obj = windows()
df_dense_rank = obj.dense_rank(df_new)
df_rank = obj.rank(df_new)
df_row_number = obj.row_number(df_new)
df_dense_rank.display()
df_rank.display()
df_row_number.display()


In [0]:
df.write.format('delta').mode('overwrite').option('header','true').option('path','abfss://silver@adls5485.dfs.core.windows.net/orders').save()

In [0]:
%sql

create table if not exists sales_prj.gold.orders
using delta
location 'abfss://silver@adls5485.dfs.core.windows.net/orders'